In [24]:
import sys
sys.path.append('../../scripts/')
from query import *
import pandas as pd


In [25]:
df_HWO = pd.read_excel('../../data/HWO_target_list_164.xlsx')
df_HWO

df_HWO['hd_name'] = df_HWO['hd_name'].fillna('').str.replace(r'\s+', '', regex=True)

filename = '../../data/HWO_target_list_164_update.xlsx'
df_HWO.to_excel(filename, index=False)

df_HWO

,hip_name,hip_compname,hd_name,hr_name,gj_name,constellation,hostname,sy_dist,ra,dec,sy_plx,sy_vmag,st_spectype,st_teff,st_tefferr,st_lum,st_rad,st_mass
0,HIP 42438,NaN,HD72905,HR 3391,GJ 311,3 UMa,3 Ursae Majoris A,144388,1297987692,650209064.0,692576,5630,G0.5V Fe-0.5,5893,14.0,-0.011800,946,01:08:00
1,HIP 44897,NaN,HD78366,HR 3625,GJ 334.2,NaN,HD 78366,189498,1372127933,338822184.0,527709,5962,G0IV-V,5992,20.0,0.714583,1045,01:08:00
2,HIP 45038,A,HD78154A,HR 3616 A,GJ 335A,13 UMa A,13 Ursae Majoris A,205169,1375981111,671340172.0,487404,4809,F7V,6325,42.0,4.233333,1680,01:21:00
3,HIP 48113,NaN,HD84737,HR 3881,GJ 368,NaN,HD 84737,188226,1471473806,460210074.0,531277,5086,G0(V),5893,13.0,3.100000,1604,01:08:00
4,HIP 51459,A,HD90839,HR 4112 A,GJ 395,36 UMa A,36 Ursae Majoris A,129452,1576565803,559805388.0,772485,4820,F8V,6164,21.0,1.427083,1111,01:18:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,HIP 23693,NaN,HD33262A,HR 1674 A,GJ 189,zet Dor A,Zeta Doradus A,116927,763777320,-574727047.0,855233,4701,F9V Fe-0.5,6158,23.0,1.154167,1064,01:14:00
160,HIP 69965,A,HD125276A,HR 5356 A,GJ 9476A,NaN,HD 125276 A,179888,2147537320,-258154250.0,555900,5872,F9V Fe-1.5 CH-0.7,6120,41.0,0.570833,978,01:11:00
161,HIP 910,NaN,HD693,HR 33,GJ 10,6 Cet,6 Ceti,188861,28160752,-154679779.0,529489,4895,F8V Fe-0.8 CH-0.5,6190,13.0,3.310417,1505,01:18:00
162,HIP 112447,A,HD215648A,HR 8665,GJ 872A,xi Peg A,Xi Pegasi A,161509,3416732494,121728858.0,619161,4200,F6V,6193,23.0,4.566667,1852,01:25:00


In [30]:
merged_df = pd.read_excel('../../results/combined_query_with_mass_detection_limit.xlsx', dtype={'source_id': str, 'source_id_dr2': str, 'source_id_dr3': str, 'HIP Number': str})
merged_df[['HD Number 1', 'HD Number 2']] = merged_df['HD Number'].str.split(', ', expand=True, n=1)
# Clean up 'HD Number 1' and 'HD Number 2' by removing extra spaces after 'HD'
merged_df['HD Number 1'] = merged_df['HD Number 1'].str.replace(r'HD\s+', 'HD', regex=True)
merged_df['HD Number 2'] = merged_df['HD Number 2'].fillna('').str.replace(r'HD\s+', 'HD', regex=True)

merged_df = merged_df[merged_df['HZ Detection Limit [M_Earth]'] < 1.5]
merged_df

,source_id,source_id_dr2,source_id_dr3,RA,DEC,V_mag,Phot G Mean Mag,Phot BP Mean Mag,Phot RP Mean Mag,BP-RP,...,RV precision [m/s],HZ Detection Limit [M_Earth],Spectral Type,HD Number,GJ Number,HIP Number,Object Type,HZ Detection Limit Simplified [M_Earth],HD Number 1,HD Number 2
2,2683023811628007296,2683023811628007296,2683023811628007296,330.540802,1.398992,9.136045,8.403359,9.387209,7.420015,1.967194,...,0.178149,0.795958,M,HD 209290,GJ 846,108782,HighPM*,0.795956,HD209290,
9,5970137861534914304,5970137861534914304,5970137861534914304,254.727158,-39.556574,10.480717,9.771798,10.732088,8.756553,1.975535,...,0.333617,1.383546,unknown,HD 153026B,GJ 646 B,NaN,HighPM*,1.383541,HD153026B,
10,5946986063684477056,5946986063684477056,5946986063684477056,263.807237,-48.678777,10.221318,9.293614,10.480064,8.208224,2.271840,...,0.296742,0.954569,M,NaN,GJ 680,86057,HighPM*,0.954566,NaN,
11,3828238392559860992,3828238392559860992,3828238392559860992,153.072937,-3.746748,9.233858,8.331761,9.491674,7.257185,2.234489,...,0.188832,0.631374,M,NaN,GJ 382,49986,RSCVnV*,0.631373,NaN,
12,1450067137649449728,1450067137649449728,NaN,202.943071,29.276211,12.007520,10.607384,12.281963,9.379468,2.902495,...,0.677486,1.376924,NaN,NaN,NaN,NaN,NaN,1.376915,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4149,5291028181119851776,5291028181119851776,5291028181119851776,119.450120,-60.302542,5.442085,5.442085,5.732447,4.987916,0.744531,...,0.106439,1.303926,F,"HD 65907, HD 65907A",GJ 294 A,38908,HighPM*,1.303924,HD65907,HD65907A
4171,1460229442589223424,1460229369573307264,1460229442589223424,197.964282,27.882105,4.093279,4.093279,4.387678,3.625619,0.762059,...,0.057469,0.702377,F,HD 114710,GJ 502,64394,HighPM*,0.702377,HD114710,
4176,6371557530914670976,6371557530914670976,6371557530914670976,317.350227,-73.174545,5.512941,5.512941,5.813269,5.041137,0.772132,...,0.110628,1.378904,F,HD 200525,"GJ 9719 A, GJ 818.1 B, GJ 818.1 A",104440,**,1.378901,HD200525,
4237,5808612830236138368,5808612830236138368,5808612830236138368,247.119879,-70.083910,4.743968,4.743968,5.039831,4.303140,0.736691,...,0.079276,0.963608,F,HD 147584,GJ 624,80686,SB*,0.963606,HD147584,


In [31]:
# ---------------------------------------------------------------- #
#  Merge
# ---------------------------------------------------------------- #
# Perform left merges on various columns and combine results
merge_keys = ['HD Number 1', 'HD Number 2']
merged_HWO = pd.concat([df_HWO.merge(merged_df, left_on='hd_name', right_on=key, how='left') for key in merge_keys])

# Sort the combined DataFrame by 'source_id' to prioritize non-null values
merged_HWO.sort_values(by='source_id', ascending=False, inplace=True)

# Remove duplicate entries based on 'star_ID  ', keeping the first occurrence
merged_HWO.drop_duplicates(subset='hd_name', keep='first', inplace=True)

# Reset the index of the final DataFrame
merged_HWO.reset_index(drop=True, inplace=True)

# Save the final DataFrame to an Excel file
# filename = '../../results/merged_HWO.xlsx'
filename = '../../results/merged_HWO_Me<1.5.xlsx'
merged_HWO.to_excel(filename, index=False)
adjust_column_widths(filename)
